In [1]:
import pandas as pd
from datetime import datetime
import requests
import time
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
from datetime import datetime
import requests
import time
from bs4 import BeautifulSoup

In [2]:
columns = ['title', 'content', 'author', 'url' ,'date', 'year', 'month', 'day', 'regions', 'tags', 'summary', 'og_link', 'original_lang', 'label']
#df_real= pd.DataFrame(columns = columns)
df_real = pd.read_csv("real.csv", index_col=0)
df_real

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,"Ukraine war: Zelensky says 31,000 troops kille...","Volodymyr Zelensky says 31,000 Ukrainian soldi...",Kathryn Armstrong,https://bbc.com/news/world-europe-68397525,2024-02-25 00:00:00,2024.0,2.0,25.0,NaN,War in Ukraine,"Volodymyr Zelensky says 31,000 Ukrainian soldi...",https://bbc.com/news/world-europe-68397525,en,0
1,Ukrainian woman gives Welsh pub new lease of life,A woman who fled war-torn Ukraine hopes that r...,Andrew Arthur,https://bbc.com/news/articles/cj7vlvy1yvro,2024-03-05 00:00:00,2024.0,3.0,5.0,NaN,War in Ukraine,A woman who fled war-torn Ukraine hopes that r...,https://bbc.com/news/articles/cj7vlvy1yvro,en,0
2,Refugee returned to Kyiv due to state of UK home,A Ukrainian refugee returned to live in Kyiv r...,Gemma Dunstan,https://bbc.com/news/articles/c84jp1g2j71o,2024-03-14 00:00:00,2024.0,3.0,14.0,NaN,War in Ukraine,A Ukrainian refugee returned to live in Kyiv r...,https://bbc.com/news/articles/c84jp1g2j71o,en,0
3,The unlikely species entangled in Ukraine's re...,"Lawsuits, art and the sandy blind mole-rat hav...",Darya Tsymbalyuk,https://bbc.com/future/article/20240223-ukrain...,2024-02-24 00:00:00,2024.0,2.0,24.0,NaN,War in Ukraine,"Lawsuits, art and the sandy blind mole-rat hav...",https://bbc.com/future/article/20240223-ukrain...,en,0
4,Ukraine war: Vatican envoy called in over Pope...,Ukraine has summoned the Vatican's envoy after...,Kathryn Armstrong,https://bbc.com/news/world-europe-68538267,2024-03-11 00:00:00,2024.0,3.0,11.0,NaN,War in Ukraine,Ukraine has summoned the Vatican's envoy after...,https://bbc.com/news/world-europe-68538267,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6623,Democrats have the power but the insurrectioni...,As the U.S. braced for the possibility of more...,Jaisal Noor,https://therealnews.com/democrats-have-the-pow...,2021-01-15 00:00:00,2021.0,1.0,15.0,NaN,NaN,,https://therealnews.com/democrats-have-the-pow...,English,0
6624,Neo-fascists found their voice in Trump. Where...,As the U.S. braced for the possibility of more...,Marc Steiner,https://therealnews.com/neo-fascists-found-the...,2021-01-15 00:00:00,2021.0,1.0,15.0,NaN,NaN,,https://therealnews.com/neo-fascists-found-the...,English,0
6625,One frightening story reveals the true cost of...,As the U.S. braced for the possibility of more...,Taya Graham,https://therealnews.com/one-frightening-story-...,2021-01-14 00:00:00,2021.0,1.0,14.0,NaN,NaN,,https://therealnews.com/one-frightening-story-...,English,0
6626,"Yes, this kind of thing happens in America","On Monday, congressional Democrats began an at...",Jaisal Noor,https://therealnews.com/yes-this-kind-of-thing...,2021-01-12 00:00:00,2021.0,1.0,12.0,NaN,NaN,,https://therealnews.com/yes-this-kind-of-thing...,English,0


In [4]:
from re import I


content = ""

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

def get_urls(url): #több oldalon keresztül van 10 link -> ezeket kell kiszedni, mert azokon vannak a hamis hírek
    case_links= []  # The URL of the page with the list of disinformation cases
  # Send a GET request
    response = requests.get(url, headers=headers)
  #print(response.text)
  # Parse the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    #print(response.text)

  # Find links to individual disinformation cases
  # Assuming each case is linked using <a> tags within divs of a specific class
  # This selector might need adjustment based on actual page structure
 
    div_element = soup.find_all('h2', class_="entry-title")
    print(len(div_element))

    # Alternatively, if class names are dynamic, consider using other attributes or combinations
    # div_element = soup.find('div', attrs={"data-component-name": "card"})

    if div_element:
        for divs in div_element:
    # Extract the link from the 'data-open-link' attribute
            link = divs.find('a')
            case_links.append(link)
            
    else:
      print('error getting div')

    return case_links

In [5]:
def get_data(start_page):
    end_page = 50
    df_real_temp = pd.DataFrame(columns=columns)
    page = start_page
    case_links = []
    while page <= (start_page + end_page):
        print(page)
        data_list = []
        try:
            time.sleep(5)  # Be polite and don't hammer the server
            if page == 1:
                case_links = get_urls("https://therealnews.com/latest-news")
                #print(case_links)
            else:
                url_ = f"https://therealnews.com/latest-news/page/{page}"
                #print(url_)
                case_links = get_urls(url_)

            for link in case_links:
            
                time.sleep(1)
                # Safety net for missing href attributes
                case_url = link.get('href', "")
                print(case_url)
                if case_url:
                    
                    try:
                        case_response = requests.get(case_url, headers=headers)
                        case_soup = BeautifulSoup(case_response.text, 'html.parser')
                        #print(case_soup)
                        #------------------------------------------------------------------------
                        title_element = case_soup.find('h1', class_='entry-title entry-title--with-subtitle')

                        target_div = case_soup.find('div', class_ = 'entry-content')

                        if target_div:
                            paragraph_texts = []

                            # Iterate over each 'p' tag within the div
                            
                            for element in target_div.next_elements:
                                if element.name == "aside":
                                    # Stop if an <aside> tag is encountered
                                    break
                                if element.name == "p":
                                    paragraph_texts.append(element.get_text())
                            
                                    content = ' '.join(paragraph_texts)

                        else:
                            content = "Could not find content."

                        title = title_element.text.strip() if title_element else 'Title not found'
                        
        
                        try:
                            author = case_soup.find('span', class_ = "author vcard").text.strip()
                        except:
                            print('No author')
                            author = ""

                        try:
                            time_ = case_soup.find('time', class_ = 'entry-date published').text.strip()
                            date = pd.to_datetime(time_)
                        except:
                            print("NO time")
                            date = 0


                        regions_text = ""
                        tags_line = ""

                      

                        
                        new_page_data = [title,content, author, case_url, date, date.year, date.month, date.day, regions_text, tags_line, " ", case_url,"English", 0]

                        print(f'Title: {title} \n Author: {author} \n Date: {date} \n Content: {content} \n \n')
                        df_real_temp = df_real_temp.append(pd.Series(new_page_data, index = df_real_temp.columns), ignore_index = True)

                       # print(data_list)
                    except Exception as e:
                        print(f"Error loading case page: {e}")
                else:
                    print("No href found for a case link.")

         
            page += 1
            print(page)
            print("\n")
        except Exception as e:
            print(f"Error processing page {page}: {e}")
    print(df_real_temp)
    return df_real_temp

In [62]:

df_real_temp = get_data(154)


154
10
https://therealnews.com/coal-miners-union-rebukes-manchin-for-effort-to-kill-build-back-better
Title: Coal miners union rebukes Manchin for effort to kill Build Back Better 
 Author: Julia Conley 
 Date: 2021-12-22 00:00:00 
 Content: This story originally appeared in Common Dreams on Dec. 21, 2021. It is shared here with permission under a Creative Commons (CC BY-NC-ND 3.0) license. The push for passage of the Build Back Better Act got a potential boost late Monday as the United Mine Workers of America, the coal miners’ union that Sen. Joe Manchin counts as a key ally, called on the right-wing Democratic senator to reconsider his opposition to his party’s signature economic agenda. UMWA president Cecil Roberts outlined several provisions in the $1.75 trillion investment in anti-poverty programs and climate action which would directly benefit the union’s 80,000 members, and made clear that coal miners are counting on the West Virginia lawmakers to “revisit his opposition.” 
 



C:\Users\annak\AppData\Local\Temp\ipykernel_2032\3409934850.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_real_temp = df_real_temp.append(pd.Series(new_page_data, index = df_real_temp.columns), ignore_index = True)


https://therealnews.com/merry-christmas-ya-tacky-animal
Title: Merry Christmas, ya tacky animal! 
 Author: Lyta Gold 
 Date: 2021-12-21 00:00:00 
 Content: Christmas is a time to be with the ones you love, to give gifts and give thanks, and to celebrate the birth of Jesus Christ (if you’re so inclined). It is also a time for gaudy decorations, Griswoldian light displays, mall Santas, and a carnivalesque orgy of capitalist commercialism, formulaic Hallmark movies, and sugary crap. Does this mean, as we hear every year, that we’ve forgotten the “true spirit” of the holiday, or is there something meaningful and worthwhile in the giant tacky spectacle? What does our attachment to the tackiest parts of Christmas say about us and our aesthetic attachment to “low culture”? Whether we’re talking about the holidays, pop music, or frosted lip gloss, it’s high time we develop a more nuanced, empathetic, and less elitist way to talk about pop culture and the politics of “good” and “bad” taste. Thi

In [63]:
df_real = df_real.append(df_real_temp, ignore_index = True)

C:\Users\annak\AppData\Local\Temp\ipykernel_2032\2677697864.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_real = df_real.append(df_real_temp, ignore_index = True)


In [64]:
df_real

,title,content,author,url,date,year,month,day,regions,tags,summary,og_link,original_lang,label
0,"Ukraine war: Zelensky says 31,000 troops kille...","Volodymyr Zelensky says 31,000 Ukrainian soldi...",Kathryn Armstrong,https://bbc.com/news/world-europe-68397525,2024-02-25 00:00:00,2024.0,2.0,25.0,NaN,War in Ukraine,"Volodymyr Zelensky says 31,000 Ukrainian soldi...",https://bbc.com/news/world-europe-68397525,en,0
1,Ukrainian woman gives Welsh pub new lease of life,A woman who fled war-torn Ukraine hopes that r...,Andrew Arthur,https://bbc.com/news/articles/cj7vlvy1yvro,2024-03-05 00:00:00,2024.0,3.0,5.0,NaN,War in Ukraine,A woman who fled war-torn Ukraine hopes that r...,https://bbc.com/news/articles/cj7vlvy1yvro,en,0
2,Refugee returned to Kyiv due to state of UK home,A Ukrainian refugee returned to live in Kyiv r...,Gemma Dunstan,https://bbc.com/news/articles/c84jp1g2j71o,2024-03-14 00:00:00,2024.0,3.0,14.0,NaN,War in Ukraine,A Ukrainian refugee returned to live in Kyiv r...,https://bbc.com/news/articles/c84jp1g2j71o,en,0
3,The unlikely species entangled in Ukraine's re...,"Lawsuits, art and the sandy blind mole-rat hav...",Darya Tsymbalyuk,https://bbc.com/future/article/20240223-ukrain...,2024-02-24 00:00:00,2024.0,2.0,24.0,NaN,War in Ukraine,"Lawsuits, art and the sandy blind mole-rat hav...",https://bbc.com/future/article/20240223-ukrain...,en,0
4,Ukraine war: Vatican envoy called in over Pope...,Ukraine has summoned the Vatican's envoy after...,Kathryn Armstrong,https://bbc.com/news/world-europe-68538267,2024-03-11 00:00:00,2024.0,3.0,11.0,NaN,War in Ukraine,Ukraine has summoned the Vatican's envoy after...,https://bbc.com/news/world-europe-68538267,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6623,Democrats have the power but the insurrectioni...,As the U.S. braced for the possibility of more...,Jaisal Noor,https://therealnews.com/democrats-have-the-pow...,2021-01-15 00:00:00,2021,1,15,,,,https://therealnews.com/democrats-have-the-pow...,English,0
6624,Neo-fascists found their voice in Trump. Where...,As the U.S. braced for the possibility of more...,Marc Steiner,https://therealnews.com/neo-fascists-found-the...,2021-01-15 00:00:00,2021,1,15,,,,https://therealnews.com/neo-fascists-found-the...,English,0
6625,One frightening story reveals the true cost of...,As the U.S. braced for the possibility of more...,Taya Graham,https://therealnews.com/one-frightening-story-...,2021-01-14 00:00:00,2021,1,14,,,,https://therealnews.com/one-frightening-story-...,English,0
6626,"Yes, this kind of thing happens in America","On Monday, congressional Democrats began an at...",Jaisal Noor,https://therealnews.com/yes-this-kind-of-thing...,2021-01-12 00:00:00,2021,1,12,,,,https://therealnews.com/yes-this-kind-of-thing...,English,0


In [65]:
df_real.to_csv("real.csv")